In [1]:
import glob
import os
import json
from os import walk
import re
import random
from subprocess import call
 

In [2]:

clssInd = {}

In [3]:
with open('clssInd.txt', 'r') as fin:    
    test_list = [row.strip() for row in fin]    
    for clss in test_list:
        # print(clss.split(' '))
        [clssName, index] = clss.split(' ')
        clssInd[clssName] = index
    print(clssInd)


{'sit': '0', 'feetup': '1', 'bodylower': '2', 'laydown': '3', 'lying': '4', 'bodyshake': '5', 'grooming': '6', 'turn': '7', 'tailing': '8', 'sitdown': '9', 'footup': '10', 'armstretch': '11', 'mounting': '12', 'taillow': '13', 'heading': '14', 'arch': '15', 'roll': '16', 'getdown': '17', 'footpush': '18', 'walkrun': '19', 'bodyscratch': '20'}


In [4]:
rawframe =[]
classes = {}
directory = '/home/oem/data/catdog'
for subdir in os.listdir(directory):    
    if subdir.startswith("2020"):
        dir = os.path.join(directory, subdir)
        for file_name in os.listdir(dir):    
            if file_name.endswith("mp4"):
                class_name = file_name.split('-')[1]                
                class_index = clssInd[class_name]
                if class_name not in classes.keys():
                    classes[class_name] = []
                classes[class_name].append(subdir + "/" + file_name[:-4])                            
                dir = subdir + "/" + file_name[:-4]
                rawframe.append(dir)                
                
                if not os.path.exists(dir):                                        
                    os.mkdir(dir)                
                    src = os.path.join(subdir, file_name)                
                    dest = os.path.join(dir, 'img_%05d.jpg')                      
                    print("extracting {}".format(src))              
                    call(["ffmpeg", "-i", src, "-threads", "16", dest])                   
    else:
        continue

In [5]:
# rawframe = []

# for subdir, dirs, files in os.walk('./'):
#     if len(re.findall('2020', subdir)) is not 0:          
        
#         for file_name in files:
#             print(file_name)
#             if file_name[-1] == '4':                            
#                 class_name = file_name.split('-')[1]
#                 class_index = clssInd[class_name]
#                 if class_name not in classes.keys():
#                     classes[class_name] = []
#                 classes[class_name].append(subdir + "/" + file_name)                            
#                 dir = subdir[2:] + "/" + file_name[:-4]
#                 rawframe.append(dir + " " + class_index)                

#                 if the dir does not exist, create it.
#                 if not os.path.exists(dir):                                        
#                     os.mkdir(dir)                
#                     src = os.path.join(subdir[2:], file_name)                
#                     dest = os.path.join(dir, 'img_%05d.jpg')  
#                     print("extracting {}".format(src))              
#                     call(["ffmpeg", "-i", src, "-threads", "16", dest])                   
                

In [7]:
# MAXCAP = 300
s_rate1 = 0.8
# s_rate2 = 0.9
trainRaw = {}
valRaw = {}          
arr = []
missarr = []
total = 0

for clss in classes:               
    cls_len = len(classes[clss])
    # cls_len = min(len(classes[clss]), MAXCAP)            
    s_point1 = int(s_rate1 * cls_len)     
    # s_point2 = int(s_rate2 * cls_len)     
    
    for item in classes[clss]:                
        if len(glob.glob(item + "/*.jpg")) == 0:            
            missarr.append(item)    
    list = [item + " {} {}".format(len(glob.glob(item + "/*jpg")), clssInd[clss]) for item in classes[clss]]
    # if (len(list) > MAXCAP):
    #     list.sort(key=lambda x: int(x.split(' ')[1]))    


    # list = list[:MAXCAP]       
    random.shuffle(list)
    trainRaw[clss] = list[:s_point1]
    valRaw[clss] = list[s_point1:]     
    # testRaw[clss] = list[s_point2:cls_len]     
    # couting each classes

    arr.append((clss, cls_len))
    total += cls_len
    # print('{} : {}, train: {}, val {}'.format(clss, len(classes[clss]), len(trainRaw[clss]), len(valRaw[clss])))         
    print('{} {}'.format(clss, len(classes[clss])))         
# print(arr)
print(total)


NameError: name 'MAXCAP' is not defined

In [7]:
len(missarr)

0

In [8]:
# for item in missarr:
#     src = item + ".mp4"
#     dest = src[:-4] + '/img_%05d.jpg'                                          
#     print("extracting {}".format(src))              
#     call(["ffmpeg", "-i", src, "-threads", "32", dest])         

In [9]:
with open('train_short300.txt', 'w') as f:
    for clss in trainRaw:
        for item in trainRaw[clss]:
            f.write("%s\n" % item)

In [10]:
with open('val_short300.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            f.write("%s\n" % item)